In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false; // disable scroll bar when displaying Folium map
}

# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Global Historical Climatology Network daily (GHCNd)](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe - it's a wonderfully large dataset to play with! In particular, you will be asked to use data from the Ann Arbor Michigan location (my home!). and this is stored in the file: `assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`

Each row in this datafile corresponds to a single observation from a weather station, and has the following variables:
* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write a python notebook which plots line graphs of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015. (Based on the graph, do you think extreme weather is getting more frequent in 2015?)
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

I've written some steps I think would be good to go through, but there are other ways to solve this assignment so feel free to explore the pandas library! What I really want to see is an image that looks like this sketch I drew at my desk:

![](assets/chris_sketch.png)

In [ ]:
#  I'll be using the folium package to render the data into a map in Jupyter.
import folium
import pandas as pd

# get the location information for this dataset
df = pd.read_csv("assets/BinSize_d400.csv")
station_locations_by_hash = df[
    df["hash"] == "fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89"
]

# get longitude and lattitude to plot
lons = station_locations_by_hash["LONGITUDE"].tolist()
lats = station_locations_by_hash["LATITUDE"].tolist()

# plot on a beautiful folium map
my_map = folium.Map(location=[lats[0], lons[0]], height=500, zoom_start=9)
for lat, lon in zip(lats, lons):
    folium.Marker([lat, lon]).add_to(my_map)

# render map in Jupyter
display(my_map)

## Step 1
Load the dataset and transform the data into Celsius (refer to documentation) then extract all of the rows which have minimum or maximum temperatures.

__hint: when I did this step I had two DataFrame objects, each with ~80,000 entries in it__

In [ ]:
import pandas as pd
import numpy as np

# Loading dataset.
df = pd.read_csv("assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv")
# Transferring Date value and converting degrees.
df["Date"] = pd.to_datetime(df["Date"])
df["Data_Value"] = df["Data_Value"] / 10
df.head()

## Step 2
In order to visualize the data we would plot the min and max data for each day of the year between the years 2005 and 2014 across all weather stations. But we also need to find out when the min or max temperature in 2015 falls below the min or rises above the max for the previous decade.

If you did step 1 you have two Series objects with min and max times for the years 2005 through 2015. You can use Pandas `groupby` to create max and min temperature Series objects across all weather stations for each day of these years, and you can deal with the records for February 29 (the leap year) by dropping them.

__hint: when I finished this step, I had two DataFrame objects, each with exactly 4015 observations in them__

In [ ]:
# Breaking the datetime column into year and dayofyear.
df["Year"] = df["Date"].dt.year
df["dayofyear"] = df["Date"].dt.dayofyear

# Adjusting dayofyear for leap years and removing Feb 29th dates data.
df["dayofyear"] = df.apply(
    lambda row: (
        row["dayofyear"] - 1
        if row["Date"].is_leap_year and row["dayofyear"] > 59
        else row["dayofyear"]
    ),
    axis=1,
)

df = df[~((df["Date"].dt.month == 2) & (df["Date"].dt.day == 29))]

# Creating two year groups for data to make creation of pivot tables easier.
df["YearGroup"] = np.where(df["Year"].between(2005, 2014), "05-14", "15")

df.head()

In [ ]:
# Creating pivot tables grouped by dayofyear values and across YearGroup and Elements (Min or Max) columns.
pivot = df.pivot_table(
    index=["dayofyear"],
    columns=["YearGroup", "Element"],
    values="Data_Value",
    aggfunc={"Data_Value": ["min", "max"]},
).reset_index()

# Renaming the columns to flatten out the pivot table and dropping columns that are not needed.
pivot.columns = [
    "DayOfYear",
    "05-14-max",
    "TMIN_max",
    "15-max",
    "TMIN_max2",
    "TMAX_min",
    "05-14-min",
    "TMAX_min2",
    "15-min",
]
pivot.drop(
    labels=["TMIN_max", "TMAX_min", "TMIN_max2", "TMAX_min2"], axis=1, inplace=True
)
pivot.head()

## Step 3
Now that you have grouped the daily max and min temperatures for each day of the years 2005 through 2015, you can separate out the data for 2015. Then you can use the Pandas `groupby` function to find the max and min of the temperature data for each __day of the year__ for the 2005-2014 data.

__hint: at the end of this step I had two DataFrames, one of maximum and the other of minimum values, which each had 365 observations in them. I also had another pair of similar DataFrames but only for the year 2015.__

In [ ]:
# Creating tables for 2015 record values that fall above or below 2005-2014 range.
min_15_record = pivot.loc[pivot["15-min"] < pivot["05-14-min"]].drop(columns=["15-max", "05-14-max"])
max_15_record = pivot.loc[pivot["15-max"] > pivot["05-14-max"]].drop(columns=["15-min", "05-14-min"])

## Step 4
Now it's time to plot! You need to explore matplotlib in order to plot line graphs of the min and max temperatures for the years 2005 through 2014 and to scatter plot __only__ the daily 2015 temperatures that exceeded those values.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from calendar import month_abbr

# Creating arbitrary middle of month days for plotting month values on the plot.
# This is instead of plotting dayofyear values.
month_mids = [pd.Timestamp(f"2023-{month:02d}-15").dayofyear for month in range(1, 13)]
month_labels = [month_abbr[month] for month in range(1, 13)]

# Creating a plot and saving the plt axis as an object for easier access to members.
plt.figure()
ax = plt.gca()

# Plotting main line charts
plt.plot(
    pivot["DayOfYear"],
    pivot["05-14-min"],
    color="blue",
    label="Lowest temperature",
)
plt.plot(
    pivot["DayOfYear"],
    pivot["05-14-max"],
    color="red",
    label="Highest temperature",
)

# Filling between the line charts.
plt.fill_between(
    pivot["DayOfYear"],
    pivot["05-14-min"],
    pivot["05-14-max"],
    facecolor="grey",
    alpha=0.3,
)

# Plotting the scatter plots for 2015 record values.
plt.scatter(
    min_15_record["DayOfYear"],
    min_15_record["15-min"],
    s=20,
    color="black",
    label="2015 record",
)

# Setting plot parameters.
plt.ylim(min_15_record["15-min"].min() - 10, pivot["05-14-max"].max() + 10)
plt.scatter(max_15_record["DayOfYear"], max_15_record["15-max"], s=20, color="black")
plt.xticks(ticks=month_mids, labels=month_labels) # We create custom x-axis values for months.
ax.yaxis.set_major_locator(MultipleLocator(10))
ax.yaxis.set_minor_locator(MultipleLocator(5))
plt.legend()
plt.title(
    label="Ann-Arbor Record Lowest and Highest Temperatures for 2005-2014 Period and 2015 Records"
)
plt.xlabel(xlabel="Month of Year")
plt.ylabel(ylabel="Temperature (°C)")
plt.grid(visible=True, axis="y")

# Removing the boxes from the graph.
# You can use ax.get_children() to examine different children values that can be manipulated.
for value in ax.spines.values():
    value.set_visible(False)